<a href="https://colab.research.google.com/github/MatiasMoreno707/lab12-lp/blob/main/lab12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### a. Realice el preprocesamiento de la información que incluya el análisis de datos faltantes,tratamiento de outliers a nivel univariado y multivariado, y balanceo de clases.

In [2]:
#!pip install category-encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.5 MB/s eta 0:00:00


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
import category_encoders as ce
from collections import Counter

In [17]:
df = pd.read_csv('obesidad.csv')
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Normal_Weight
1,Female,21,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Normal_Weight
3,Male,27,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Overweight_Level_I
4,Male,22,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Overweight_Level_II


In [21]:
df.shape

(2111, 17)

In [18]:
df.dtypes

,0
Gender,object
Age,int64
Height,float64
Weight,float64
family_history_with_overweight,object
FAVC,object
FCVC,int64
NCP,int64
CAEC,object
SMOKE,object


In [19]:
df.isnull().sum()

,0
Gender,0
Age,0
Height,0
Weight,0
family_history_with_overweight,0
FAVC,0
FCVC,0
NCP,0
CAEC,0
SMOKE,0


In [15]:
num_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

In [20]:
#método de IQR
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower_bound, lower_bound,
                       np.where(df[col] > upper_bound, upper_bound, df[col]))

#### b. Convierta las variables categóricas a dummies por medio de la función ‘OneHotEncoder’ de la librería ‘category_encoders’ la cual debe ser previamente instalada mediante la sentencia ‘!pip install category-encoders’.

In [24]:
# Conversión de variables categóricas a dummies
encoder = ce.OneHotEncoder(use_cat_names=True)
X = encoder.fit_transform(df.drop('NObeyesdad', axis=1))

#### c. Separe los datos en entrenamiento (80%) y prueba (20%), realice un Naive Bayes mediante la función ‘GaussianNB’ y ‘MultinomialNB’. Compare los accuracy para los resultados obtenidos y diga con cuál de ellos se obtuvo un mejor desempeño.


In [25]:
y = df['NObeyesdad']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [27]:
# GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)

In [28]:
# MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_mnb = mnb.predict(X_test)
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)

In [29]:
# Resultados
print(f"Accuracy GaussianNB: {accuracy_gnb:.2f}")
print(f"Accuracy MultinomialNB: {accuracy_mnb:.2f}")

Accuracy GaussianNB: 0.53
Accuracy MultinomialNB: 0.57


In [30]:
if accuracy_gnb > accuracy_mnb:
    print("GaussianNB tiene mejor desempeño.")
elif accuracy_gnb < accuracy_mnb:
    print("MultinomialNB tiene mejor desempeño.")
else:
    print("Ambos modelos tienen el mismo desempeño.")

MultinomialNB tiene mejor desempeño.
